# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. 

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. 

Для решения поставленной задачи необходимо будет:

1. Загрузить и изучить данные
2. Попробуем доказать предположение, что умножение исходной матрицы на случайную квадратичную обратимую матрицу не изменить качество предсказания по ее признакам, использую данные в задании формулы логистической регрессии.
3. Преобразуем модель, обучим ее и сравним метрики предсказаний базовых и преобразованных данных

Загрузим необходимые для проведения исследований библиотеки.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score


## Загрузка данных

Посмотрим на имеющиеся данные.

In [ ]:
try:
    data = pd.read_csv('insurance.csv')
except:
    data = pd.read_csv('/datasets/insurance.csv')
data

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


Хорошо читаемые данные. Всего 5 столбцов и 5000 строк. Очевидно целевым признаком является столбец "Страховые выплаты", остальные признаки, по условию задачи необходимо изменить, чтобы их было трудно восстановить.

Посмотрим состав данных

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Пропусков нет, все данные имеют числовой формат подходящий для использования линейной регрессии.

Не вижу смысл искать и удалять дубликаты, так как исходных признаков очень мало и вполне вероятно простое совпадение данных клиентов.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Для подтверждения вышеуказанного суждения попробуем доказать, что предсказания без преобразований равны предсказаниям с преобразованиями (a=a')

Из условия задачи нам известно, что формула обучения для линейной регрессии

$$
w = (X^T X)^{-1} X^T y
$$

Тогда для преобразованной базы данных:

$$
w = (P^TX^T XP)^{-1} (XP)^T y
$$

Преобразуем формулу, чтобы лучше читалась

$$
w = (P^T(X^T X)P)^{-1} P^T X^T y
$$

В первой скобке мы получили три матрицы. Раскроем скобки используя правило раскрытия скобок при операции с матрицами

$$
w = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

Сократим

$$
w = P^{-1} (X^T X)^{-1} X^T y
$$

Так как

$$
w = (X^T X)^{-1} X^T y
$$

То:

$$
w'=P^{-1} w
$$

И так как формула предсказания:

$$
a=Xw
$$

То формула для преобразованной матрицы:

$$
a'=XPw'
$$

Подставим вместо w' выражение P^{-1}w

$$
a'=XP P^{-1}w
$$

То есть:

$$
a'=Xw
$$

Что и требовалось доказать.

**Ответ:** Качество модели после преобразования не изменится.

**Обоснование:** предсказания исходной матрицы = предсказаниям исходной матрицы домноженной на обратимую матрицу

## Алгоритм преобразования

**Алгоритм**



Первым делом разделим признаки.

In [ ]:
X = data.drop('Страховые выплаты', axis=1)
y = data['Страховые выплаты']

Для преобразования признаков попробуем умножить датафрейм, содержащий их, на случайную квадратичную матрицу P. Для этого создадим такую матрицу с количеством строк равному количеству признаков.

In [ ]:
r = np.random.RandomState(42)
P = r.rand(4, 4)
P

array([[0.37454012, 0.95071431, 0.73199394, 0.59865848],
       [0.15601864, 0.15599452, 0.05808361, 0.86617615],
       [0.60111501, 0.70807258, 0.02058449, 0.96990985],
       [0.83244264, 0.21233911, 0.18182497, 0.18340451]])

Проверим матрицу на обратимость. Для этого зададим обратную ей матрицу P_inv.

In [ ]:
P_inv = np.linalg.inv(P)
P_inv

array([[-0.32076901, -0.12766508,  0.06141427,  1.32518674],
       [ 0.35151041, -1.88500014,  1.65560045, -1.0003883 ],
       [ 1.14080312,  1.3467702 , -2.0407373 ,  0.70794071],
       [-0.08202687,  1.42666425, -0.17238177, -0.10600441]])

In [ ]:
def error_inv(X):
    try: 
        np.linalg.inv(X)
        print('Матрица обратима')
    except:
        return error_inv(X)

In [ ]:
error_inv(P)

Матрица обратима


Не уверен, что получилось правильно

Попробуем привести получаемую матрицу в более читабельный вид

In [ ]:
(P @ P_inv).astype(int)

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

In [ ]:
np.allclose(np.dot(P, P_inv), np.eye(P.shape[0]))


True

In [ ]:
np.allclose(np.dot(P_inv, P), np.eye(P.shape[0]))

True

Полученная случайная матрица обратима.

Приступим к преобразованию данных

In [ ]:
X_new = X @ P
X_new

,0,1,2,3
0,29822.908329,35127.958687,1024.286164,48143.823952
1,22850.379746,26914.146043,785.064454,36896.601889
2,12627.939787,14874.047975,433.958805,20393.226004
3,25071.437266,29530.327057,859.956818,40463.797343
4,15693.844869,18486.012841,539.613636,25339.498732
...,...,...,...,...
4995,21465.839326,25282.983552,736.856437,34650.401463
4996,31504.563692,37108.519229,1080.784169,50852.909647
4997,20382.584156,24007.204956,699.339679,32897.634320
4998,19662.765162,23158.992905,675.668272,31736.256913


**Обоснование**

Предсказания по преобразованному датафрейму с большой степенью вероятности будут соответствовать предсказания базового, так как мы доказали, что умножение исходной матрицы на обратимую квадратичную матрицу не изменит качество предсказания.

## Проверка алгоритма

Обучим модель (по условию задачи у нас линейная регрессия) и определим качество модели в соответствии с метрикой R2/

In [ ]:
model = LinearRegression()
model.fit(X, y)

predictions = model.predict(X) 
print('Метрика R2 для неизмененной базы данных составляет:', r2_score(y, predictions))

Метрика R2 для неизмененной базы данных составляет: 0.4249455028666801


И теперь обучим модель, использую преобразованные данные, и найдем её качество

In [ ]:
model.fit(X_new, y)

predictions_new = model.predict(X_new) 
print('Метрика R2 для измененной базы данных составляет:', r2_score(y, predictions_new))

Метрика R2 для измененной базы данных составляет: 0.42494550286667976


Качество модели при использовании преобразованных данных соответствует качеству модели с базовыми данными. 

In [ ]:
X_new @ P_inv  

,0,1,2,3
0,1.000000e+00,41.0,49600.0,1.000000e+00
1,8.833155e-13,46.0,38000.0,1.000000e+00
2,-9.161878e-13,29.0,21000.0,-4.797520e-12
3,-1.007047e-12,21.0,41700.0,2.000000e+00
4,1.000000e+00,28.0,26100.0,-5.092805e-12
...,...,...,...,...
4995,-1.723248e-12,28.0,35700.0,2.000000e+00
4996,1.169080e-12,34.0,52400.0,1.000000e+00
4997,-1.250771e-12,20.0,33900.0,2.000000e+00
4998,1.000000e+00,22.0,32700.0,3.000000e+00
